In [ ]:
import xarray as xa
from matplotlib import pyplot as plt
import cartopy
import glob
import re
natsort = lambda s: [int(t) if t.isdigit() else t.lower() for t in re.split("(\d+)", s)]
import xcdat as xc
import numpy as np
from matplotlib import colors
import matplotlib as mpl
mpl.rc('font', family='DejaVu Serif') 
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
def get_forced(var, std=False):
    if var=='tos':
        cat = 'Omon'
        start_year = 1950
        vid = 'tos'
    elif var=='pr':
        cat = 'Amon'
        start_year = 1979
        vid = 'pr'
    elif var=='monmaxpr':
        cat = 'Aday'
        start_year = 1979
        vid = 'pr'
    root_dir = "/p/lustre3/shiduan/ForceSMIP"  # path to forcesmip data (NCAR)
    models = ['CESM2', 'MIROC6', 'MPI-ESM1-2-LR', 'MIROC-ES2L', 'CanESM5']
    reference_period = (str(start_year)+"-01-01", "2022-12-31") # climatological period (for anomaly calculations)
    tv_time_period = (str(start_year)+"-01-01", "2023-01-01")
    model_mean_list = []
    all_data = {}
    for model in models:
        model_data = []
        print(model)
        mpath = root_dir + '/Training/' + cat + '/' + var + '/' + model
        mfiles = glob.glob(mpath + '/*')
        # print(mfiles)
        if model == "CESM2":
            members = [p.split("ssp370_")[-1].split(".1880")[0] for p in mfiles]
        else:
            members = [p.split("_")[-1].split(".")[0] for p in mfiles]
        members.sort(key=natsort)
        # initialize model ensemble xarray dataset
        ds_model = None
        for im, member in enumerate(members):
            # print member progress
            print('.', end='')
            # get member filename
            fn = glob.glob(mpath + "/*_" + member + ".*.nc")
            # make sure filename is unique
            if len(fn) != 1:
                raise ValueError("Unexpected number of model members")
            else:
                fn = fn[0]
            # load data
            ds = xc.open_dataset(fn)
            ds = ds.bounds.add_missing_bounds(axes=['T'])
            # remove singletons / lon
            ds = ds.squeeze()
            #ds = ds.drop_vars('lon')
            # subset data to user-specified time period
            ds = ds.sel(time=slice(tv_time_period[0], tv_time_period[1]))
            # calculate departures (relative to user-specified reference time period)
            ds = ds.temporal.departures(vid, freq='month', reference_period=reference_period)
            if std:
                ds = ds.groupby(ds.time.dt.month)/ds.groupby(ds.time.dt.month).std(dim='time')
            if 'file_qf' in ds.variables:
                ds = ds.drop('file_qf')
            if 'ref_time' not in locals():
                ref_time = ds.time
            for i, t in enumerate(ds.time.values):
                m = t.month; y = t.year
                rt = ref_time.values[i]; rm = rt.month; ry = rt.year
                if ((ry != y) | (rm != m)):
                    raise ValueError("model time and reference time do not match")
            ds["time"] = ref_time.copy()
            # add model realization to model ensemble dataset
            if ds_model is None:
                ds_model = ds
            else:
                ds_model = xa.concat((ds_model, ds), dim='member')
        all_data[model] = ds_model.load()
        # ds_model_mean = ds_model.mean(dim='member', skipna=False)
        # ds_model_mean = ds_model_mean.load()
        # all_models.append(ds_model) 
        # model_mean_list.append(ds_model_mean)
    return all_data

In [ ]:
tos_all_data = get_forced(var='tos', std=False)
tos_all_data_std = get_forced(var='tos', std=True)

In [ ]:
pr_all_data = get_forced(var='pr', std=False)
pr_all_data_std = get_forced(var='pr', std=True)

In [ ]:
monmaxpr_all_data = get_forced(var='monmaxpr', std=False)
monmaxpr_all_data_std = get_forced(var='monmaxpr', std=True)

In [ ]:
def get_snr(model, all_data, vid):
    data_in = all_data[model]
    # Forced Trend
    data_forced = data_in.mean(dim='member')
    time_length = len(data_forced.time.data)
    time_step = np.linspace(1, time_length+1, time_length)
    data_forced['time'] = time_step
    trend_forced = data_forced[vid].polyfit(dim='time', deg=1, skipna=False)
    trend_forced = trend_forced.polyfit_coefficients.sel(degree=1)*12*10 # degree per decade
    # individual trend
    trend_all = []
    for i in range(len(data_in.member)):
        realization = data_in[vid].isel(member=i)
        time_length = len(data_forced.time.data)
        time_step = np.linspace(1, time_length+1, time_length)
        realization['time'] = time_step
        trend_realization = realization.polyfit(dim='time', deg=1, skipna=False)
        trend_realization = trend_realization.polyfit_coefficients.sel(degree=1)*12*10 # degree per decade
        trend_all.append(trend_realization)
    trend_all_xa = xa.concat(trend_all, dim='ens')
    print(trend_all_xa.shape)
    trend_std = trend_all_xa.std(dim='ens')
    return trend_forced, trend_all, trend_std

# TOS

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CESM2', tos_all_data_std, vid='tos')
trend_forced, trend_all, trend_std = get_snr('CESM2', tos_all_data, vid='tos')
snr_cesm2_tos = np.abs(trend_forced)/trend_std
snr_std_cesm2_tos = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CanESM5', tos_all_data_std, vid='tos')
trend_forced, trend_all, trend_std = get_snr('CanESM5', tos_all_data, vid='tos')
snr_canesm5_tos = np.abs(trend_forced)/trend_std
snr_std_canesm5_tos = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC6', tos_all_data_std, vid='tos')
trend_forced, trend_all, trend_std = get_snr('MIROC6', tos_all_data, vid='tos')
snr_MIROC6_tos = np.abs(trend_forced)/trend_std
snr_std_MIROC6_tos = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MPI-ESM1-2-LR', tos_all_data_std, vid='tos')
trend_forced, trend_all, trend_std = get_snr('MPI-ESM1-2-LR', tos_all_data, vid='tos')
snr_mpi_tos = np.abs(trend_forced)/trend_std
snr_std_mpi_tos = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC-ES2L', tos_all_data_std, vid='tos')
trend_forced, trend_all, trend_std = get_snr('MIROC-ES2L', tos_all_data, vid='tos')
snr_miroc_es2l_tos = np.abs(trend_forced)/trend_std
snr_std_miroc_es2l_tos = np.abs(trend_forced_std)/trend_std_std

# Pr

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CESM2', pr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('CESM2', pr_all_data, vid='pr')
snr_cesm2_pr = np.abs(trend_forced)/trend_std
snr_std_cesm2_pr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CanESM5', pr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('CanESM5', pr_all_data, vid='pr')
snr_canesm5_pr = np.abs(trend_forced)/trend_std
snr_std_canesm5_pr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC6', pr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MIROC6', pr_all_data, vid='pr')
snr_MIROC6_pr = np.abs(trend_forced)/trend_std
snr_std_MIROC6_pr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MPI-ESM1-2-LR', pr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MPI-ESM1-2-LR', pr_all_data, vid='pr')
snr_mpi_pr = np.abs(trend_forced)/trend_std
snr_std_mpi_pr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC-ES2L', pr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MIROC-ES2L', pr_all_data, vid='pr')
snr_miroc_es2l_pr = np.abs(trend_forced)/trend_std
snr_std_miroc_es2l_pr = np.abs(trend_forced_std)/trend_std_std

# Monmaxpr

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CESM2', monmaxpr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('CESM2', monmaxpr_all_data, vid='pr')
snr_cesm2_monmaxpr = np.abs(trend_forced)/trend_std
snr_std_cesm2_monmaxpr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('CanESM5', monmaxpr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('CanESM5', monmaxpr_all_data, vid='pr')
snr_canesm5_monmaxpr = np.abs(trend_forced)/trend_std
snr_std_canesm5_monmaxpr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC6', monmaxpr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MIROC6', monmaxpr_all_data, vid='pr')
snr_MIROC6_monmaxpr = np.abs(trend_forced)/trend_std
snr_std_MIROC6_monmaxpr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MPI-ESM1-2-LR', monmaxpr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MPI-ESM1-2-LR', monmaxpr_all_data, vid='pr')
snr_mpi_monmaxpr = np.abs(trend_forced)/trend_std
snr_std_mpi_monmaxpr = np.abs(trend_forced_std)/trend_std_std

In [ ]:
trend_forced_std, trend_all_std, trend_std_std = get_snr('MIROC-ES2L', monmaxpr_all_data_std, vid='pr')
trend_forced, trend_all, trend_std = get_snr('MIROC-ES2L', monmaxpr_all_data, vid='pr')
snr_miroc_es2l_monmaxpr = np.abs(trend_forced)/trend_std
snr_std_miroc_es2l_monmaxpr = np.abs(trend_forced_std)/trend_std_std

# Plot local SNR

In [ ]:
import seaborn as sns

In [ ]:
sns.color_palette("rocket_r", as_cmap=True)

In [ ]:
xx, yy = np.meshgrid(snr_canesm5_pr.lon, snr_canesm5_pr.lat)
print(xx.shape)
snr_canesm5_pr.shape

In [ ]:
condition = snr_canesm5_pr > 2
snr_canesm5_pr_filter = xa.where(snr_canesm5_pr > 2, 1, np.NAN)
snr_canesm5_pr_filter.plot()

In [ ]:
fig = plt.figure(figsize=(7, 7))
cmap = sns.color_palette("rocket_r", as_cmap=True)
norm_pr = colors.Normalize(vmin=0, vmax=6)
norm_pr_diff = colors.TwoSlopeNorm(vmin=-1.8, vcenter=0, vmax=1.8)
cmap_diff = 'coolwarm'
ax = fig.add_subplot(211, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 2, 100000], hatches=[None, '\/.',], colors='none')

ax = fig.add_subplot(212, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 2, 100000], hatches=[None, '\/.',], colors='none',)


In [ ]:
fig = plt.figure(figsize=(14, 5))
cmap = sns.color_palette("rocket_r", as_cmap=True)
norm_pr = colors.Normalize(vmin=0, vmax=6)
norm_pr_diff = colors.TwoSlopeNorm(vmin=-1.8, vcenter=0, vmax=1.8)
cmap_diff = 'coolwarm'
ax = fig.add_subplot(3, 6, 1, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CanESM5 monpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_canesm5_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_canesm5_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '/',], colors='none')'''
ax = fig.add_subplot(3, 6, 7, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '/',], colors='none')'''

ax = fig.add_subplot(3, 6, 13, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_pr-snr_canesm5_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 2, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CESM2 monpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_cesm2_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_cesm2_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\/',], colors='none')'''
ax = fig.add_subplot(3, 6, 8, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\/',], colors='none')'''
ax = fig.add_subplot(3, 6, 14, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_pr-snr_cesm2_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 3, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC6 monpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_MIROC6_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_MIROC6_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\/',], colors='none')'''
ax = fig.add_subplot(3, 6, 9, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\\//',], colors='none')'''
ax = fig.add_subplot(3, 6, 15, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_pr-snr_MIROC6_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 4, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC-ES2L monpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\\//',], colors='none')'''
ax = fig.add_subplot(3, 6, 10, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
'''ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), 
            levels=[0, 1, 10000], hatches=[None, '\\//',], colors='none')'''
ax = fig.add_subplot(3, 6, 16, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_pr-snr_miroc_es2l_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 5, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MPI-ESM1-2-LR monpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_mpi_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 11, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 17, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_pr-snr_mpi_pr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 6, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MultiModel')
ave1 = (snr_mpi_pr+snr_cesm2_pr+snr_MIROC6_pr+snr_miroc_es2l_pr+snr_canesm5_pr)/5
ave2 = (snr_std_mpi_pr+snr_std_cesm2_pr+snr_std_MIROC6_pr+snr_std_miroc_es2l_pr+snr_std_canesm5_pr)/5
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 12, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 18, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2-ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)
plt.tight_layout()

cbar_ax = fig.add_axes([1.02, 0.55, 0.01, 0.3])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr, cmap=cmap), cax=cbar_ax, extend='both', label='SNR')
cbar_ax = fig.add_axes([1.02, 0.1, 0.01, 0.2])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr_diff, cmap=cmap_diff), cax=cbar_ax, extend='both', label='SNR Difference')
plt.savefig('localSNR-pr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 5))
cmap = sns.color_palette("rocket_r", as_cmap=True)
norm_pr = colors.Normalize(vmin=0, vmax=6)
norm_pr_diff = colors.TwoSlopeNorm(vmin=-1.5, vcenter=0, vmax=1.5)
cmap_diff = 'coolwarm'
ax = fig.add_subplot(3, 6, 1, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CanESM5 monmaxpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_canesm5_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 7, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 13, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_monmaxpr-snr_canesm5_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 2, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CESM2 monmaxpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_cesm2_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 8, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 14, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_monmaxpr-snr_cesm2_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 3, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC6 monmaxpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_MIROC6_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 9, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 15, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_monmaxpr-snr_MIROC6_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 4, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC-ES2L monmaxpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_miroc_es2l_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 10, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 16, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_monmaxpr-snr_miroc_es2l_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 5, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MPI-ESM1-2-LR monmaxpr')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_mpi_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 11, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 17, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_monmaxpr-snr_mpi_monmaxpr, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 6, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MultiModel')
ave1 = (snr_mpi_monmaxpr+snr_cesm2_monmaxpr+snr_MIROC6_monmaxpr+snr_miroc_es2l_monmaxpr+snr_canesm5_monmaxpr)/5
ave2 = (snr_std_mpi_monmaxpr+snr_std_cesm2_monmaxpr+snr_std_MIROC6_monmaxpr+snr_std_miroc_es2l_monmaxpr+snr_std_canesm5_monmaxpr)/5
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 12, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 18, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2-ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)
plt.tight_layout()

cbar_ax = fig.add_axes([1.02, 0.55, 0.01, 0.3])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr, cmap=cmap), cax=cbar_ax, extend='both', label='SNR')
cbar_ax = fig.add_axes([1.02, 0.1, 0.01, 0.2])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr_diff, cmap=cmap_diff), cax=cbar_ax, extend='both', label='SNR Difference')
plt.savefig('localSNR-monmaxpr.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 5))
cmap = sns.color_palette("rocket_r", as_cmap=True)
norm_pr = colors.Normalize(vmin=0, vmax=20)
norm_pr_diff = colors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=10)
cmap_diff = 'coolwarm'
ax = fig.add_subplot(3, 6, 1, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CanESM5 tos')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_canesm5_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 7, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 13, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_canesm5_tos-snr_canesm5_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 2, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('CESM2 tos')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_cesm2_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 8, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 14, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_cesm2_tos-snr_cesm2_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 3, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC6 tos')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_MIROC6_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 9, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 15, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_MIROC6_tos-snr_MIROC6_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 4, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MIROC-ES2L tos')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_miroc_es2l_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 10, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 16, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_miroc_es2l_tos-snr_miroc_es2l_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 5, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MPI-ESM1-2-LR tos')
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_mpi_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 11, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 17, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, snr_std_mpi_tos-snr_mpi_tos, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)

ax = fig.add_subplot(3, 6, 6, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.set_title('MultiModel')
ave1 = (snr_mpi_tos+snr_cesm2_tos+snr_MIROC6_tos+snr_miroc_es2l_tos+snr_canesm5_tos)/5
ave2 = (snr_std_mpi_tos+snr_std_cesm2_tos+snr_std_MIROC6_tos+snr_std_miroc_es2l_tos+snr_std_canesm5_tos)/5
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 12, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2, transform=cartopy.crs.PlateCarree(), norm=norm_pr, cmap=cmap)
ax = fig.add_subplot(3, 6, 18, projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.3)
ax.contourf(snr_canesm5_pr.lon, snr_canesm5_pr.lat, ave2-ave1, transform=cartopy.crs.PlateCarree(), norm=norm_pr_diff, cmap=cmap_diff)
plt.tight_layout()

cbar_ax = fig.add_axes([1.02, 0.55, 0.01, 0.3])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr, cmap=cmap), cax=cbar_ax, extend='both', label='SNR')
cbar_ax = fig.add_axes([1.02, 0.1, 0.01, 0.2])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm_pr_diff, cmap=cmap_diff), cax=cbar_ax, extend='both', label='SNR Difference')
plt.savefig('localSNR-tos.png', dpi=150, bbox_inches='tight')
plt.show()